In [1]:
%load_ext dotenv
%dotenv

In [6]:
# get raw data from GCP bucket
import pandas as pd
import os

# pull in and explore meta data
KEY = 'all_sources_metadata_2020-03-13.csv'
df = pd.read_csv('gs://{}/{}'.format(os.environ['BUCKET_NAME'], KEY), encoding='utf-8', low_memory=False)

In [56]:
df.head(2)

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
0,c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,angiotensinconverting enzyme ace as a sarscov...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"zhang, haibo penninger, josef m li, yimin zhon...",Intensive Care Med,2.002765e+09,#3252,True
1,53eccda7977a31e3d0f565c884da036b1e85438e,CZI,comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"cao, yanan li, lin feng, zhimin wan, shengqing...",Cell Discovery,3.003431e+09,#1861,True


In [57]:
df.source_x.value_counts(dropna=False)

PMC        27337
CZI         1236
biorxiv      566
medrxiv      361
Name: source_x, dtype: int64

In [58]:
# clean numbers and special characters
regex_arr = ['\d','\t','\r','\)','\(','\/', ':', ';', '&', '#', '-', '\.']
def clean_numbers_and_special_characters (df, col):
    df[col] = df[col].replace(regex=regex_arr, value='')
    df[col] = df[col].str.rstrip()
    df[col] = df[col].str.lower()
    return df    

df = clean_numbers_and_special_characters(df, 'title')
df = clean_numbers_and_special_characters(df, 'abstract')
df = clean_numbers_and_special_characters(df, 'authors')
df.head(2)

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
0,c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,angiotensinconverting enzyme ace as a sarscov...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"zhang, haibo penninger, josef m li, yimin zhon...",Intensive Care Med,2.002765e+09,#3252,True
1,53eccda7977a31e3d0f565c884da036b1e85438e,CZI,comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"cao, yanan li, lin feng, zhimin wan, shengqing...",Cell Discovery,3.003431e+09,#1861,True
